## Double-loop framework implementation with Fossil Case (using integrated ultra-supercritical power plant model)


This notebook presents an implementation of the double-loop framework for the fossil case study using the integrated ultra-supercritical power plant model with a molten-salt based thermal energy storage.

First, import all necessary modules and packages.

In [1]:
# Import Python library packages
from types import ModuleType
import pandas as pd
from importlib import resources

# Import Pyomo packages
import pyomo.environ as pyo

# Import IDAES packages
from idaes.apps.grid_integration import Tracker
from idaes.apps.grid_integration import Bidder
from idaes.apps.grid_integration import PlaceHolderForecaster
from idaes.apps.grid_integration import DoubleLoopCoordinator
from idaes.apps.grid_integration.model_data import GeneratorModelData

# Import Prescient simulator
from prescient.simulator import Prescient

# Import integrated ultra-supercritical power plant with energy storage model
from multiperiod_double_loop_usc import MultiPeriodUsc


Interactive Python mode detected; using default matplotlib backend for plotting.


Set up fossil generator characteristics using a dictionary and double-loop model parameters. The fossil generator is the integrated ultra-supercritical power plant. The double-loop model parameters are the number of hours for the bidding and tracking models, number of scenarios to use, and the number of days for simulation.

In [2]:
generator_data = {
    "gen_name": "102_STEAM_3",
    "generator_type": "thermal",
    "p_min": 286,
    "p_max": 460,
    "min_down_time": 4,
    "min_up_time": 8,
    "ramp_up_60min": 60,
    "ramp_down_60min": 60,
    "shutdown_capacity": 286,
    "startup_capacity": 286,
    "production_cost_bid_pairs": [
        (286, 22.16602),
        (350, 19.0723),
        (400, 18.29703),
        (430, 17.71727),
        (460, 17.71727),
    ],
    "startup_cost_pairs": [(4, 7355.42), (10, 10488.35), (12, 11383.41)],
    "fixed_commitment": 1,
}
model_data = GeneratorModelData(**generator_data)

tracking_horizon = 10  # hours
bidding_horizon = 48  # hours
n_scenario = 2  # for bidding
n_tracking_hour = 1  # advance n_tracking_hour (i.e. assume we solve every hour)
num_days = 2


Set up the solver and the forecaster objects.

In [3]:
# create solver
solver = pyo.SolverFactory("ipopt")
solver.options = {
    "max_iter": 200,
}

# create forecaster object
with resources.open_text("idaes.apps.grid_integration.examples", "lmp_forecasts_concat.csv") as f:
    price_forecasts_df = pd.read_csv(f)
forecaster = PlaceHolderForecaster(price_forecasts_df=price_forecasts_df)



Set up the objects for Tracker, Bidder, and Coordinator.

In [4]:
# Tracker
mp_usc_tracker = MultiPeriodUsc(
    horizon=tracking_horizon,
    model_data=model_data
)

thermal_tracker = Tracker(
    tracking_model_object=mp_usc_tracker,
    n_tracking_hour=n_tracking_hour,
    solver=solver,
)

# Projection Tracker
mp_usc_projection_tracker = MultiPeriodUsc(
    horizon=tracking_horizon,
    model_data=model_data
)

thermal_projection_tracker = Tracker(
    tracking_model_object=mp_usc_projection_tracker,
    n_tracking_hour=n_tracking_hour,
    solver=solver,
)

# Bidder
mp_usc_bidder = MultiPeriodUsc(
    horizon=bidding_horizon,
    model_data=model_data
)
thermal_bidder = Bidder(
    bidding_model_object=mp_usc_bidder,
    n_scenario=n_scenario,
    solver=solver,
    forecaster=forecaster,
)

# Coordinator
coordinator = DoubleLoopCoordinator(
    bidder=thermal_bidder,
    tracker=thermal_tracker,
    projection_tracker=thermal_projection_tracker,
)


>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time period
>>> Creating USC model and initialization for each time

Set up the plugin module needed for the Prescient simulator

In [5]:
class PrescientPluginModule(ModuleType):
    def __init__(self, get_configuration, register_plugins):
        self.get_configuration = get_configuration
        self.register_plugins = register_plugins


plugin_module = PrescientPluginModule(
    get_configuration=coordinator.get_configuration,
    register_plugins=coordinator.register_plugins,
)


Get all generators data for Prescient simulator

In [6]:
rts_gmlc_data_dir = "C:\\grid\\source_code\\Prescient\\downloads\\rts_gmlc\\RTS-GMLC\\RTS_Data\\SourceData"


Declare the options for Prescient simulator

In [7]:
options = {
    "data_path": rts_gmlc_data_dir,
    "input_format": "rts-gmlc",
    "simulate_out_of_sample": True,
    "run_sced_with_persistent_forecast_errors": True,
    "output_directory": "bidding_plugin_test_multiperiod_rankine",
    "start_date": "2020-07-10",
    "num_days": num_days,
    "sced_horizon": tracking_horizon,
    "ruc_horizon": bidding_horizon,
    "compute_market_settlements": True,
    "day_ahead_pricing": "LMP",
    "ruc_mipgap": 0.01,
    "symbolic_solver_labels": True,
    "reserve_factor": 0.0,
    "deterministic_ruc_solver": "gurobi",
    "output_ruc_solutions": True,
    "sced_solver": "gurobi",
    "print_sced": True,
    "enforce_sced_shutdown_ramprate": True,
    "plugin": {
        "doubleloop": {
            "module": plugin_module,
            "bidding_generator": "102_STEAM_3",
        }
    },
}


Run Prescient simulator.

In [ ]:
Prescient().simulate(**options)


Initializing simulation...


C:\Users\susarlan\Anaconda3\envs\gmlc\lib\site-packages\egret\parsers\rts_gmlc\parser.py:256: FutureWarning: 
        Use pd.to_datetime instead.

  df = pd.read_csv(file_name,
C:\Users\susarlan\Anaconda3\envs\gmlc\lib\site-packages\egret\parsers\rts_gmlc\parser.py:256: FutureWarning: 
        Use pd.to_datetime instead.

  df = pd.read_csv(file_name,
C:\Users\susarlan\Anaconda3\envs\gmlc\lib\site-packages\egret\parsers\rts_gmlc\parser.py:256: FutureWarning: 
        Use pd.to_datetime instead.

  df = pd.read_csv(file_name,
C:\Users\susarlan\Anaconda3\envs\gmlc\lib\site-packages\egret\parsers\rts_gmlc\parser.py:256: FutureWarning: 
        Use pd.to_datetime instead.

  df = pd.read_csv(file_name,
C:\Users\susarlan\Anaconda3\envs\gmlc\lib\site-packages\egret\parsers\rts_gmlc\parser.py:304: FutureWarning: 
        Use pd.to_datetime instead.

  df = pd.read_csv(file_name,
C:\Users\susarlan\Anaconda3\envs\gmlc\lib\site-packages\egret\parsers\rts_gmlc\parser.py:304: FutureWarning: 
     

Setting default t0 state in RTS-GMLC parser


Dates to simulate: 2020-07-10 to 2020-07-11
RUC activation hours: 0
Final RUC date: 2020-07-11
Using current day's forecasts for RUC solves
Using persistent forecast error model when projecting demand and renewables in SCED


Extracting scenario to simulate
Ipopt 3.13.2: max_iter=200


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientif

Calculating PTDF Matrix Factorization


...

  14  6.1725969e+08 8.54e+04 8.39e+02  -1.0 1.90e+08    -  5.90e-02 1.20e-01f  1
  15  6.1655563e+08 8.52e+04 8.73e+02  -1.0 1.98e+08    -  2.94e-01 1.63e-03f  1
  16  6.0597963e+08 8.37e+04 8.53e+02  -1.0 3.01e+08    -  3.43e-02 1.83e-02f  1
  17  6.0107192e+08 8.28e+04 8.34e+02  -1.0 3.05e+08    -  5.28e-02 1.07e-02f  1
  18  5.9424166e+08 8.15e+04 8.12e+02  -1.0 3.03e+08    -  5.47e-02 1.54e-02f  1
  19  5.8408864e+08 7.93e+04 7.81e+02  -1.0 3.35e+08    -  6.89e-02 2.83e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  5.6442785e+08 1.05e+05 7.44e+02  -1.0 3.78e+08    -  4.41e-02 4.63e-02f  1
  21  5.6370341e+08 1.05e+05 7.30e+02  -1.0 3.78e+08    -  1.72e-02 1.67e-03f  1
  22  5.5899343e+08 1.04e+05 5.50e+02  -1.0 3.63e+08    -  3.80e-01 1.24e-02f  1
  23  5.4921799e+08 1.28e+05 5.47e+02  -1.0 9.27e+08    -  1.15e-02 1.96e-02f  1
  24  5.4911906e+08 1.28e+05 2.74e+02  -1.0 5.25e+07    -  6.82e-01 3.48e-03f  1
  25  5.4904117e+08 3.24e+04

  23  5.8588788e+08 1.19e+05 4.58e+02  -1.0 4.03e+08    -  6.44e-02 1.38e-02f  1
  24  5.8571083e+08 1.16e+05 6.28e+02  -1.0 1.61e+07    -  7.40e-01 2.12e-02f  1
  25  5.8550554e+08 6.51e+05 1.57e+02  -1.0 4.87e+06    -  4.53e-01 9.46e-01h  1
  26  5.8461384e+08 2.78e+06 7.16e+01  -1.0 9.75e+06    -  6.61e-01 1.00e+00f  1
  27  5.8120171e+08 4.06e+07 4.79e+01  -1.0 3.70e+07    -  3.28e-01 1.00e+00f  1
  28  5.7741378e+08 4.04e+07 3.24e+01  -1.0 5.80e+07    -  3.31e-01 7.33e-01f  1
  29  5.7736262e+08 4.00e+07 3.23e+01  -1.0 4.48e+07    -  4.62e-03 9.74e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30  5.7317215e+08 1.60e+08 6.81e+01  -1.0 1.53e+09    -  1.47e-04 2.86e-02f  1
  31  5.7312345e+08 1.60e+08 6.47e+01  -1.0 2.07e+08    -  2.01e-01 2.18e-03h  1
  32  5.7306527e+08 1.56e+08 6.35e+01  -1.0 3.21e+07    -  6.69e-01 2.08e-02h  1
  33  5.7285317e+08 5.86e+07 2.81e+01  -1.0 3.25e+07    -  6.58e-01 6.19e-01h  1
  34  5.7274847e+08 4.31e+07

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  7.3590419e+08 2.38e+03 1.19e+01  -1.0 9.86e+06    -  8.67e-02 1.12e-01f  1
  11  7.2578420e+08 3.87e+03 1.45e+02  -1.0 1.08e+07    -  4.55e-01 3.41e-01f  1
  12  7.1719902e+08 5.44e+03 2.01e+02  -1.0 1.94e+07    -  2.56e-01 1.45e-01f  1
  13  7.1703441e+08 5.43e+03 5.82e+02  -1.0 2.24e+07    -  7.88e-01 2.36e-03f  1
  14  7.1112199e+08 5.29e+03 4.81e+02  -1.0 8.77e+07    -  2.98e-01 2.57e-02f  1
  15  7.0166861e+08 5.66e+03 4.61e+02  -1.0 1.35e+08    -  6.59e-02 3.57e-02f  1
  16  6.7341685e+08 4.17e+04 4.25e+02  -1.0 1.49e+08    -  4.32e-03 1.01e-01f  1
  17  6.6223869e+08 4.56e+04 3.81e+02  -1.0 1.54e+08    -  2.97e-01 3.88e-02f  1
  18  6.6144682e+08 4.55e+04 3.78e+02  -1.0 2.21e+08    -  8.57e-02 2.32e-03f  1
  19  6.5515884e+08 4.47e+04 4.41e+02  -1.0 2.26e+08    -  1.52e-01 1.87e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  6.3337227e+08 7.25e+04

   9  7.3458940e+08 9.34e+03 9.97e+00  -1.0 6.15e+05    -  9.86e-01 4.86e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  7.3237906e+08 9.18e+03 1.57e+01  -1.0 3.61e+07    -  2.31e-02 1.92e-02f  1
  11  7.2805297e+08 8.91e+03 1.98e+01  -1.0 4.16e+07    -  4.12e-02 3.80e-02f  1
  12  7.1582212e+08 8.67e+03 3.92e+02  -1.0 4.46e+07    -  3.61e-01 1.04e-01f  1
  13  7.1550486e+08 8.65e+03 5.23e+02  -1.0 5.34e+07    -  1.68e-01 2.07e-03f  1
  14  7.1273224e+08 8.48e+03 7.85e+02  -1.0 5.88e+07    -  4.90e-01 2.44e-02f  1
  15  7.0343613e+08 1.04e+04 6.88e+02  -1.0 1.30e+08    -  1.85e-01 5.25e-02f  1
  16  6.7829154e+08 6.69e+04 6.46e+02  -1.0 1.71e+08    -  2.79e-03 1.12e-01f  1
  17  6.7795658e+08 6.68e+04 6.16e+02  -1.0 1.41e+08    -  2.61e-01 1.55e-03f  1
  18  6.4274104e+08 2.43e+05 5.10e+02  -1.0 2.11e+08    -  2.34e-02 1.60e-01f  1
  19  6.4236221e+08 2.42e+05 5.07e+02  -1.0 2.17e+08    -  1.69e-01 1.66e-03f  1
iter    objective    inf_pr 

   2  6.1696309e+08 9.98e+04 9.98e+01  -1.0 3.76e+05    -  4.86e-04 1.76e-03h  1
   3  6.1751399e+08 9.94e+04 9.99e+01  -1.0 2.14e+05    -  2.20e-03 3.47e-03h  1
   4  6.1866646e+08 9.88e+04 9.94e+01  -1.0 1.51e+05    -  5.83e-03 6.80e-03h  1
   5  6.3519148e+08 8.91e+04 1.07e+02  -1.0 1.24e+05    -  1.44e-02 9.79e-02h  1
   6  6.4129126e+08 8.55e+04 9.78e+01  -1.0 1.16e+05    -  6.70e-02 4.02e-02h  1
   7  7.2180808e+08 3.84e+04 1.39e+02  -1.0 1.16e+05    -  2.19e-02 5.51e-01h  1
   8  7.3111892e+08 3.29e+04 3.26e+01  -1.0 7.82e+04    -  9.79e-01 1.43e-01h  1
   9  7.4120337e+08 2.47e+04 2.45e+01  -1.0 3.46e+06    -  4.11e-01 2.48e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  7.4151758e+08 2.43e+04 6.39e+02  -1.0 5.83e+06    -  4.15e-01 1.70e-02h  1
  11  7.4205665e+08 2.11e+04 6.88e+02  -1.0 9.96e+06    -  2.68e-01 1.32e-01h  1
  12  7.4206667e+08 2.09e+04 1.08e+03  -1.0 1.35e+07    -  6.16e-01 1.02e-02h  1
  13  7.3531640e+08 1.87e+04